# Motor Loss Table Generation
This tool is used for motor loss table generation for GEN5SW

In [ ]:
import os
import glob
import numpy as np
import pandas as pd 
import math
import array as arr
from scipy.interpolate import griddata
from bokeh.plotting import figure, show
from bokeh.palettes import Category10
from bokeh.models import BoxAnnotation
from bokeh.io import output_notebook
output_notebook()

In [ ]:
output_notebook()
plot_tools = ("crosshair","hover","box_zoom","wheel_zoom","pan","reset","save")

In [ ]:
#Functions, may move to seperate library?
def peak_2_rms(value_peak):
    value_rms = value_peak / math.sqrt(2)
    return value_rms

def rms_2_peak(value_rms):
    value_peak = value_rms * math.sqrt(2)
    return value_peak

def rpm_2_rads(speed_rpm):
    speed_rads = speed_rpm * 2 * math.pi 
    return speed_rads

def eff_pc(output_value, input_value):
   eff_pc = (output_value/input_value) * 100
   return eff_pc

In [ ]:
# Set directory of data
path = r'/Users/sfreedman/Desktop/Run_1/220V/Q1'

# Join path and look for any .csv's
all_files = glob.glob(os.path.join(path, "*Signals*.csv"))

# Create dataframe from all_files csv's
df = pd.concat( (pd.read_csv(f) for f in all_files), ignore_index=True) 

In [ ]:
#Rename  signals in dataframe to something readable.
df.rename(columns = {
                    ' TesOp_B.L2m_TarTrq_IOP'                   :'T_Demanded',
                    ' tesOutputData.L2mTes_EstTrq.val_IOP'      :'T_Estimated',
                    'Transducer_Torque_IOP'                     :'T_Measured',
                    'Transducer_Speed_IOP'                      :'N_Measured',
                    ' tesInputData.L2mPosSpdArb_RotorSpd_IOP'   :'N_Unit_rpm',
                    ' sensvdcOutputData.L2mSensVdc_Vdc.val_IOP' :'V_DC',
                    ' sensidcOutputData.L2mSensIdc_Idc.val_IOP' :'I_DC',
                    ' posSpdArbInputData.L2mSensVmt_Vd_IOP'     :'Ud_RMS',
                    ' posSpdArbInputData.L2mSensVmt_Vq_IOP'     :'Uq_RMS',
                    ' tesOutputData.L2mTes_Id.val_IOP'          :'Id_Peak',
                    ' tesOutputData.L2mTes_Iq.val_IOP'          :'Iq_Peak',
                    'InverterEfficiency_IOP'                    :'Loss_Inv_Unit'
                    }, inplace = True)

#Delete unwanted coloumns to speed up processing.
df.drop(columns = {
                    ' TesOp_B.L2m_TarTrq_MCP',
                    ' tesOutputData.L2mTes_EstTrq.val_MCP',
                    'Transducer_Torque_MCP','Transducer_Speed_MCP',
                    ' tesInputData.L2mPosSpdArb_RotorSpd_MCP',
                    ' sensvdcOutputData.L2mSensVdc_Vdc.val_MCP',
                    ' sensidcOutputData.L2mSensIdc_Idc.val_MCP',
                    ' posSpdArbInputData.L2mSensVmt_Vd_MCP',
                    ' posSpdArbInputData.L2mSensVmt_Vq_MCP',
                    ' tesOutputData.L2mTes_Id.val_MCP',
                    ' tesOutputData.L2mTes_Iq.val_MCP',
                    'InverterEfficiency_MCP', 
                    'E2E Value',
                    'timestamp'
                    }, inplace=True)


In [ ]:
# Post process signals

# Mechanical
df["N_Measured_rads"]   = rpm_2_rads(df["N_Measured"])
df["P_Measured"]        = df["N_Measured_rads"] * df["T_Measured"]

# DC Link
df["P_DC"]              = df["V_DC"] * df["I_DC"]

# Phase Peak 
df["Ud_Peak"]           = rms_2_peak(df["Ud_RMS"]) 
df["Uq_Peak"]           = rms_2_peak(df["Uq_RMS"]) 
  
# Phase RMS
df["Id_RMS"]            = peak_2_rms(df["Id_Peak"]) 
df["Iq_RMS"]            = peak_2_rms(df["Iq_Peak"]) 
  
# Phase 
df["P_Ph"]              = (3/2) * ( ( df["Ud_Peak"] * df["Id_Peak"] ) + ( df["Uq_Peak"] * df["Iq_Peak"] ) ) 

# Losses
df["Loss_Inv_Real"]     = df["P_DC"] - df["P_Ph"]
df["Loss_Mtr"]          = df["P_Measured"] - df["P_Ph"]
df["Loss_Mtr_Comp"]     = df["Loss_Mtr"] - df["Loss_Inv_Unit"]
df["Loss_Sys"]          = df["Loss_Inv_Real"] - df["Loss_Mtr"]


In [ ]:
# Plot test points
# could be faster by rounding or finding unique values?

test_points_plot =  figure(
                            toolbar_location  = "right",
                            tools = plot_tools,
                            title = "Test Points"
                            )  

test_points_plot.x(df["N_Measured"], df["T_Demanded"], size=10, alpha=1.0)

test_points_plot.xaxis.axis_label = "Speed (rpm)"
test_points_plot.yaxis.axis_label = "Torque (Nm)"

show(test_points_plot)

In [ ]:
# Plot Losses
loss_plot =  figure(
                            toolbar_location  = "right",
                            tools = plot_tools,
                            title = "Losses (W)",
                            )  

loss_plot.line(x = df.index.values, y = df["Loss_Mtr"], line_color = Category10[10][0], legend_label = "Motor Losses (W)")
loss_plot.line(x = df.index.values, y = df["Loss_Inv_Real"], line_color = Category10[10][1], legend_label = "Inverter Calcualted Losses (W)")
loss_plot.line(x = df.index.values, y = df["Loss_Inv_Unit"], line_color = Category10[10][2], legend_label = "Inverter Losses (W)")
loss_plot.line(x = df.index.values, y = df["Loss_Sys"], line_color = Category10[10][3], legend_label = "System Losses (W)")

loss_plot.xaxis.axis_label = "Sample (n)"
loss_plot.yaxis.axis_label = "Losses (W)"

show(loss_plot)

In [ ]:
# Plot Powers
powers_plot =  figure(
                        toolbar_location  = "right",
                        tools = plot_tools,
                        title = "Power (W)",
                        )  

powers_plot.line(x = df.index.values, y = df["P_DC"], line_color = Category10[10][0], legend_label = "DC Link Power (W)")
powers_plot.line(x = df.index.values, y = df["P_Ph"], line_color = Category10[10][1], legend_label = "Inverter AC Power (W)")
powers_plot.line(x = df.index.values, y = df["P_Measured"], line_color = Category10[10][2], legend_label = "Mechanical Power (W)")

powers_plot.xaxis.axis_label = "Sample (n)"
powers_plot.yaxis.axis_label = "Power (W)"

show(powers_plot)

In [ ]:

Speed_Threshold = 0
Torque_Threshold = 0

# Determine operating quadrant
df["Operating_Quadrant"] = 'VEH_STP'
df["Operating_Quadrant"] = np.where((df["N_Measured"] > Speed_Threshold) & (df["T_Measured"] > Torque_Threshold), 'DRV_FWD', df["Operating_Quadrant"])
df["Operating_Quadrant"] = np.where((df["N_Measured"] > Speed_Threshold) & (df["T_Measured"] < Torque_Threshold), 'BRK_FWD', df["Operating_Quadrant"])
df["Operating_Quadrant"] = np.where((df["N_Measured"] < Speed_Threshold) & (df["T_Measured"] < Torque_Threshold), 'DRV_REV', df["Operating_Quadrant"])
df["Operating_Quadrant"] = np.where((df["N_Measured"] < Speed_Threshold) & (df["T_Measured"] > Torque_Threshold), 'BRK_REV', df["Operating_Quadrant"])

# Efficiency
df["Eff_Inv"] = '0'
np.where( (df["Operating_Quadrant"] == 'DRV_FWD') | (df["Operating_Quadrant"] == 'DRV_REV') , eff_pc(df["P_Ph"], df["P_DC"]), df["Eff_Inv"] )
np.where( (df["Operating_Quadrant"] == 'BRK_FWD') | (df["Operating_Quadrant"] == 'BRK_REV') , eff_pc(df["P_DC"], df["P_Ph"]), df["Eff_Inv"] )

df["Eff_Mtr"] = '0'
np.where( (df["Operating_Quadrant"] == 'DRV_FWD') | (df["Operating_Quadrant"] == 'DRV_REV') , eff_pc(df["P_Measured"], df["P_Ph"]), df["Eff_Mtr"] )
np.where( (df["Operating_Quadrant"] == 'BRK_FWD') | (df["Operating_Quadrant"] == 'BRK_REV') , eff_pc(df["P_Ph"], df["P_Measured"]), df["Eff_Mtr"] )

df["Eff_Sys"] = '0'
np.where( (df["Operating_Quadrant"] == 'DRV_FWD') | (df["Operating_Quadrant"] == 'DRV_REV') , eff_pc(df["P_Measured"], df["P_DC"]), df["Eff_Sys"] )
np.where( (df["Operating_Quadrant"] == 'BRK_FWD') | (df["Operating_Quadrant"] == 'BRK_REV') , eff_pc(df["P_DC"], df["P_Measured"]), df["Eff_Sys"] )

In [ ]:
Dwell_Period = 500

df["Step_Change"]   = '0'
df["Step_Change"]   = df["T_Demanded"].diff()

Step_index = ( df.index[df['Step_Change'] != 0]-1 )
Stop_index = Step_index + Dwell_Period

# Plot transient removal sample
transient_sample = df.iloc[Step_index[3]-250 : Stop_index[3]+250]

transient_plot =  figure(
                            toolbar_location  = "right",
                            tools = plot_tools,
                            title = "Transient Removal Example",
                            )

#Annoate where the removal will occur in the data
transient_plot.add_layout( BoxAnnotation( left=Step_index[3], right = Stop_index[3], hatch_color="red",hatch_pattern="/", hatch_alpha=0.1, fill_alpha=0.2, fill_color='red' ) )

transient_plot.line(x = transient_sample.index.values, y = transient_sample["T_Demanded"], line_color = Category10[10][0], legend_label = "Demanded Torque (Nm)")
transient_plot.line(x = transient_sample.index.values, y = transient_sample["T_Estimated"], line_color = Category10[10][1], legend_label = "Estimated Torque (Nm)")
transient_plot.line(x = transient_sample.index.values, y = transient_sample["T_Measured"], line_color = Category10[10][2], legend_label = "Measured Torque (Nm)")

show(transient_plot)

#Transient Removal
delete_slice = np.array(0)

for x in range( len(Step_index) ):
    temp_slice = np.arange(Step_index[x], Stop_index[x])
    delete_slice = np.append(delete_slice, temp_slice)

delete_slice = abs(delete_slice)
df.drop(index = delete_slice)

In [ ]:
# Round measured speed to the nearest 100 rpm.
df["N_Measured_Rounded"] = df["N_Measured"].round(-2)

# Group the data in the dataframe by the measured speed (rounded) and torque demanded.
# Get average of all data within those subgroups and create new dataframe, df_steady_state.
df_steady_state = df.groupby(["N_Measured_Rounded", "T_Demanded"], as_index=False).agg("mean")

In [ ]:
# Setup a meshgrid for interpolation of loss results.
Table_Size_x        = 26
Table_Size_y        = 17
Max_Motor_Torque    = 300
Max_Motor_Speed     = 13000

Torque_BP       = np.linspace(0, Max_Motor_Torque, Table_Size_x)
Speed_BP    = np.array([0, 500, 1000, 2000, 3000, 4000, 5000, 5800, 7200, 8300, 9000, 10000, 11000, 12000])

Meshgrid_Speed_Array  = np.linspace(0, Max_Motor_Speed+1, Max_Motor_Speed+1)
Meshgrid_Torque_Array = np.linspace(0, Max_Motor_Torque+1, Max_Motor_Torque+1)

Meshgrid_Speed, Meshgrid_Torque = np.meshgrid(Meshgrid_Speed_Array, Meshgrid_Torque_Array)

# Determine operating quadrant
df_steady_state["Operating_Quadrant"] = 'VEH_STP'
df_steady_state["Operating_Quadrant"] = np.where((df_steady_state["N_Measured"] > Speed_Threshold) & (df_steady_state["T_Measured"] > Torque_Threshold), 'DRV_FWD', df_steady_state["Operating_Quadrant"])
df_steady_state["Operating_Quadrant"] = np.where((df_steady_state["N_Measured"] > Speed_Threshold) & (df_steady_state["T_Measured"] < Torque_Threshold), 'BRK_FWD', df_steady_state["Operating_Quadrant"])
df_steady_state["Operating_Quadrant"] = np.where((df_steady_state["N_Measured"] < Speed_Threshold) & (df_steady_state["T_Measured"] < Torque_Threshold), 'DRV_REV', df_steady_state["Operating_Quadrant"])
df_steady_state["Operating_Quadrant"] = np.where((df_steady_state["N_Measured"] < Speed_Threshold) & (df_steady_state["T_Measured"] > Torque_Threshold), 'BRK_REV', df_steady_state["Operating_Quadrant"])

# Rearrange data depending on quadrant.
if (df_steady_state["Operating_Quadrant"].any() == 'BRK_REV'):
    Meshgrid_Speed  = Meshgrid_Speed    *-1
    Speed_BP        = Speed_BP          *-1

elif (df_steady_state["Operating_Quadrant"].any() == 'BRK_FWD'):
    Meshgrid_Torque = Meshgrid_Torque   *-1
    Torque_BP       = Torque_BP         *-1

elif (df_steady_state["Operating_Quadrant"].any() == 'DRV_REV'):
    Meshgrid_Torque = Meshgrid_Torque   *-1
    Torque_BP       = Torque_BP         *-1
    Meshgrid_Speed  = Meshgrid_Speed    *-1
    Speed_BP        = Speed_BP          *-1

else:
    pass

MotorPowerLoss = pd.DataFrame(griddata( df_steady_state.loc[ :, ['N_Measured_Rounded','T_Measured'] ], df_steady_state["Loss_Mtr_Comp"], (Meshgrid_Speed, Meshgrid_Torque), method="linear" ))

Loss_Mtr_Table = MotorPowerLoss.iloc[Torque_BP, Speed_BP]

